In [1]:
import ee
import geemap
print ("OK!")

OK!


In [2]:
ee.Authenticate()

Enter verification code: 4/1AfJohXl-4BcW50VU_lppM9Q4crwGS9nMQzhPwJnHK2KAonlAMmlVglbJBos

Successfully saved authorization token.


In [3]:
ee.Initialize()
print ("OK!")

OK!


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_doiqkQG3NJ1t8IS?source=API


In [70]:
def s2_cloudmask(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask)).eq(0)
    return image.updateMask(mask).divide(10000)
hexi_shp = ee.FeatureCollection("users/LST_SG/HeXi")
visparam_shp = {'outline' : 0.5, 'outline_color' : 'red', 'fill_color' : 'none'}
s2_dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate("2022-06-15", "2022-08-31")\
.filterBounds(hexi_shp).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
.map(s2_cloudmask).select('B.*|MSK_CLDPRB|SCL')
visparam_image = {'min' : 0.0, 'max' : 0.3, 'bands' : ['B4', 'B3', 'B2']}
#mean = s2_dataset.mean()
#s2_map.add_layer(mean, visparam_image, 'Mean(B8, B4, B3)')
#median = s2_dataset.median()
#s2_map.add_layer(median, visparam_image, 'Medin(B8, B4, B3)')
#min_value = s2_dataset.min()
#s2_map.add_layer(min_value, visparam_image, 'Min(B8, B4, B3)')
#max_value = s2_dataset.max()
#s2_map.add_layer(max_value, visparam_image, 'Max(B8, B4, B3)')
#sum_value = s2_dataset.sum()
#s2_map.add_layer(sum_value, visparam_images, 'Min(B8, B4, B3)')
#product = s2_dataset.product()
#mode = s2_dataset.mode
count = not_cloud_dataset.count()
mosaic = not_cloud_dataset.mosaic()
s2_map = geemap.Map(center = (39.0, 103.0))
image_clip = mosaic.clip(hexi_shp)
s2_map.add_layer(hexi_shp, visparam_shp, name = "hexi_shp")
s2_map.add_layer(mosaic, visparam_image, 'RGB')
s2_map.add_layer(image_clip, visparam_image, 'RGB')
s2_map

Map(center=[39.0, 103.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [76]:
 task = ee.batch.Export.image.toDrive(
    image=moasic,
    description='image_export',
    folder='ee_demos1',
    scale=10,
    crs='EPSG:5070',
    region = hexi_shp,
)
task.start()